# Input names

In [10]:
newSceneName = "KekWait"
userHistoryEventSceneStart = f"Перешел к сцене {newSceneName}"

# Generation

In [11]:
import re

def toCamelCaseFromSnake(snake_str):
    return "".join(x.capitalize() for x in snake_str.lower().split("_"))

def toSnakeCaseFromCamel(name):
    name = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', name).lower()

def toKebabCaseFromCamel(name):
    name = re.sub('(.)([A-Z][a-z]+)', r'\1-\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1-\2', name).lower()


In [12]:
from pathlib import Path
import platform

isWindows = platform.system() == 'Windows'

def writeFileData(filePath: Path, content):
    if isWindows:
        data = content
        with filePath.open('w', encoding= 'utf-8') as file:
            file.write(data)
    else:
        data = content
        with filePath.open('w') as file:
            file.write(data)

def addLinesToFileData(filePath: Path, content):
    if isWindows:
        data = content
        with filePath.open('a', encoding= 'utf-8') as file:
            file.write(data)
    else:
        data = content
        with filePath.open('a') as file:
            file.write(data)

def getFileData(filePath: Path):
    with filePath.open() as file:
        content = file.read()
    return content

def replaceInFileData(filePath: Path, placeholder: str, newContent: str):
    fileContent = getFileData(filePath)
    fileContent = fileContent.replace(
        placeholder,
        newContent
    )
    writeFileData(filePath, fileContent)

In [13]:
sceneNameCapitaliseMask = "<#SceneNameCapitalise#>"
sceneNameCamelCaseMask = "<#SceneNameCamelCase#>"
fileContentTemplate = """import { Context } from 'telegraf'
import { Message, Update } from 'telegraf/typings/core/types/typegram'
import { SceneName } from '../enums/scene-name.enum'
import { SceneHandlerCompletion, Scene } from '../scene.interface'
import { Markup } from 'telegraf'
import { logger } from 'src/app.logger'

// =====================
// Scene data class
// =====================
interface ISceneData {
    
}

export class <#SceneNameCapitalise#> extends Scene {
    // =====================
    // Properties
    // =====================

    readonly name: SceneName = SceneName.<#SceneNameCamelCase#>

    // =====================
    // Public methods
    // =====================

    async handleEnterScene(ctx: Context<Update>): Promise<SceneHandlerCompletion> {
        logger.log(`${this.name} scene handleEnterScene. User: ${ctx.from.id} ${ctx.from.username}`)
        await this.logToUserHistory(this.historyEvent.startSceneOnboarding)
        
        await ctx.replyWithMarkdownV2(
            `Hello from *<#SceneNameCapitalise#>*`,
            super.keyboardMarkupWithAutoLayoutFor(['Hello!'])
        )

        return this.completion.inProgress(
            this.generateData({})
        )
    }

    async handleMessage(ctx: Context<Update>, dataRaw: Object): Promise<SceneHandlerCompletion> {
        logger.log(`${this.name} scene handleMessage. User: ${ctx.from.id} ${ctx.from.username}`)

        let data: ISceneData = this.restoreData(dataRaw)
        const message = ctx.message as Message.TextMessage

        await ctx.replyWithMarkdownV2(`Goodbye`, Markup.removeKeyboard())

        return this.completion.complete(SceneName.mainMenu)
    }

    async handleCallback(ctx: Context<Update>, dataRaw: Object): Promise<SceneHandlerCompletion> {
        throw new Error('Method not implemented.')
    }

    // =====================
    // Private methods
    // =====================

    private generateData(data: ISceneData): ISceneData {
        return data
    }

    private restoreData(dataRaw: object): ISceneData {
        const data: ISceneData = dataRaw as ISceneData
        return (
            data ?? this.generateData({})
        )
    }
}
"""

## Configuration

In [14]:
sceneNameCapitalise = newSceneName[0].upper() + newSceneName[1:]
sceneNameCamelCase = newSceneName[0].lower() + newSceneName[1:]
sceneNameKebabCase = toKebabCaseFromCamel(sceneNameCapitalise)

In [15]:
# Check folder path

scenesPath = Path("./src/presentation/scenes/implementations/")
creatingSceneFolder = scenesPath
creatingSceneSourceFile = creatingSceneFolder / f"{sceneNameKebabCase}.scene.ts"

userHistoryEventFilePath = Path("./src/core/user/enums/user-history-event.enum.ts")
sceneNameFilePath = Path("./src/presentation/scenes/enums/scene-name.enum.ts")
sceneFactoryFilePath = Path("./src/presentation/scenes/scene-factory.ts")


In [16]:
# Source code names update

fileContent = fileContentTemplate.replace(sceneNameCapitaliseMask, sceneNameCapitalise)
fileContent = fileContent.replace(sceneNameCamelCaseMask, sceneNameCamelCase)

## Source code generation

In [17]:
# creatingSceneFolder.mkdir(parents=True, exist_ok=True)
writeFileData(creatingSceneSourceFile, fileContent)

In [18]:
# UserHistoryEvent
placeholder = '/** New scene event placeholder */'
replaceInFileData(
    filePath=userHistoryEventFilePath,
    placeholder=placeholder,
    newContent=f"{sceneNameCamelCase} = '{userHistoryEventSceneStart}'\n\t{placeholder}"
)

# SceneName
placeholder = '/** New scene name placeholder */'
replaceInFileData(
    filePath=sceneNameFilePath,
    placeholder=placeholder,
    newContent=f"{sceneNameCamelCase} = '{sceneNameCamelCase}'\n\t{placeholder}"
)

# SceneFactory
placeholder = '/** New scene import placeholder */'
replaceInFileData(
    filePath=sceneFactoryFilePath,
    placeholder=placeholder,
    newContent="import { " + sceneNameCapitalise + " } from './implementations/" + sceneNameKebabCase + ".scene'\n" + placeholder
)

placeholder = '/** New scene generation placeholder */'
replaceInFileData(
    filePath=sceneFactoryFilePath,
    placeholder=placeholder,
    newContent=f"case SceneName.{sceneNameCamelCase}:\n\t\t\t\treturn new {sceneNameCapitalise}(configuration)\n\t\t\t{placeholder}"
)